In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

In [2]:
# Importo il csv delle fatture
df = pd.read_csv("Receipts.csv")

In [3]:
df.head()

,K_Receipt,K_Member,Quantity,Q_Amount,Q_Discount_Amount,T_Receipt
0,00010242fe8c5a6d1ba2dd792cb16214,871766c5855e863f6eccc05f988b23cb,1,58.90,NaN,2017-09-13 08:59:02
1,00018f77f2f0320c557190d7a144bdd3,eb28e67c4c0b83846050ddfb8a35d051,1,239.90,NaN,2017-04-26 10:53:06
2,000229ec398224ef6ca0657da4fc703e,3818d81c6709e39d06b2738a8d3a2474,1,199.00,NaN,2018-01-14 14:33:31
3,00024acbcdf0a6daa1e931b038114c75,af861d436cfc08b2c2ddefd0ba074622,1,12.99,NaN,2018-08-08 10:00:35
4,00042b26cf59d7ce69dfabb4e55b4fd9,64b576fb70d441e8f1b2d7d446e483c5,1,199.90,NaN,2017-02-04 13:57:51


In [4]:
# Raggruppo per K_Member e conto quanti clienti acquistano più di una volta: effettuo un count su T_Receipt
df['Count'] = df.groupby(['K_Member'])['K_Receipt'].transform('count')

In [5]:
# Cancello le colonne che non mi servono
df.drop(columns=['K_Receipt','Quantity','Q_Amount','Q_Discount_Amount'], inplace=True)

In [6]:
df.head()

,K_Member,T_Receipt,Count
0,871766c5855e863f6eccc05f988b23cb,2017-09-13 08:59:02,1
1,eb28e67c4c0b83846050ddfb8a35d051,2017-04-26 10:53:06,2
2,3818d81c6709e39d06b2738a8d3a2474,2018-01-14 14:33:31,1
3,af861d436cfc08b2c2ddefd0ba074622,2018-08-08 10:00:35,1
4,64b576fb70d441e8f1b2d7d446e483c5,2017-02-04 13:57:51,1


In [7]:
# Mi tengo le ricevute che hanno effettuato più di un acquisto
df = df[df['Count']>1]

In [8]:
# Cancello la colonna Count in quanto non significativa
df.drop(columns=['Count'], inplace=True)

In [9]:
# Resetto l'indice
df.reset_index(drop=True, inplace=True)

In [10]:
# Numero di ricevute dove i clienti hanno acquistato più di una volta
print(df.shape[0])

6159


In [11]:
# Converto 'T_Receipt' da object a datetime
df['T_Receipt'] = pd.to_datetime(df['T_Receipt'])
df.sort_values(['T_Receipt'], ascending=True, inplace=True)

In [12]:
# Calcolo la media delle differenze tra le date degli acquisti dei clienti
df['previous_receipt'] = df.groupby(['K_Member'])['T_Receipt'].shift()
df['previous_receipt'] = pd.to_datetime(df['previous_receipt'], infer_datetime_format = True)
df['diff_days'] = df['T_Receipt'] - df['previous_receipt']
df['diff_days'] = df['diff_days'].apply(lambda x: x.days)
# Li raggruppo per K_Member e faccio la media di tali differenze
data=df.groupby('K_Member', as_index=False)['diff_days'].agg('mean')
# Arrotodanmento 
data.round(decimals=0)
# Cast a int
data["diff_days"] = data["diff_days"].astype('int64')

In [13]:
data['diff_days'].mean()

80.1740473738414

In [14]:
data

,K_Member,diff_days
0,004288347e5e88a27ded2bb23747066c,170
1,004b45ec5c64187465168251cd1c9c2f,267
2,0058f300f57d7b93c477a131a59b36c3,31
3,00a39521eb40f7012db50455bf083460,10
4,00cc12a6d8b578b8ebd21ea4e2ae8b27,0
5,011575986092c30523ecb71ff10cb473,60
6,011b4adcd54683b480c4d841250a987f,176
7,012452d40dafae4df401bced74cdb490,329
8,012a218df8995d3ec3bb221828360c86,42
9,013ef03e0f3f408dd9bf555e4edcdc0a,28
